# Tarea 5: invocar el modelo de Bedrock para la generación de código

En este cuaderno, aprenderá a usar un modelo de lenguaje grande (LLM) para generar código a partir de una petición de texto.

La petición que se utiliza en este cuaderno es una sin entrenamiento previo, ya que no se proporcionan ejemplos de texto además de la petición.

A fin de demostrar la capacidad de generación de código de los modelos en Amazon Bedrock, se usa el caso práctico que se refiere a la generación de código. Analizará el uso del cliente Boto3 para comunicarse con la API de Amazon Bedrock y proporcionará a la API una entrada que consiste en una tarea, una instrucción y una entrada para que el modelo subyacente genere una salida sin proporcionar otros ejemplos. El objetivo es demostrar de qué forma el potente LLM comprende con facilidad la tarea en cuestión y genera salidas sorprendentes.


## Situación

Usted es Moe, un analista de datos en AnyCompany. La empresa desea comprender el rendimiento de ventas con respecto a diferentes productos durante el último año. Recibió un conjunto de datos llamado sales.csv. El conjunto de datos contiene las siguientes columnas:

- Date (YYYY-MM-DD) format (Formato de fecha [DD-MM-AAAA])
- Product_ID (ID_Producto) (identificador único de cada producto)
- Price (Precio) (precio al que se vende cada producto)


En este cuaderno, aprenderá a generar código para una petición determinada. Se utilizará Meta LLama 3 con la API de Amazon Bedrock con el cliente Boto3.

## Tarea 5.1: configuración del entorno

En esta tarea, establecerá el entorno.


In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## Tarea 5.2: generación de código

En esta tarea, preparará una entrada para que el servicio Amazon Bedrock genere un programa Python para su caso práctico.

#### Configuración del laboratorio: cree datos sales.csv de muestra para este laboratorio.



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

## Tarea 5.3: analizar las ventas con el programa Python generado por Amazon Bedrock

In [ ]:
# define prompt template
from langchain_core.prompts import PromptTemplate

def format_prompt(actor:str, input:str):
    match actor:
        case "user":
            prompt_template =  """<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
"""
            prompt = PromptTemplate.from_template(prompt_template)
            return prompt.format(actor=actor,input=input)
        case _:
            print("requested actor >" + actor + "< is not supported")
            return ""    

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- Total revenue by product 
- The product with the highest revenue 
- The date with the highest revenue and the revenue achieved on that date
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

"""
prompt=format_prompt("user",prompt_data)

In [ ]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 2048,
    "temperature": 0,
    "top_p": 1,
    }) 
    

## Tarea 5.4: invocar el modelo

In [ ]:
modelId = "meta.llama3-8b-instruct-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
output_list = response_body.get("generation", [])
print(output_list)

##### <i aria-hidden="true" class="far fa-copy"> </i> (Opcional) Copie el código generado de la salida impresa y ejecute el código que generó Bedrock en la celda a continuación para su validación.


Experimentó con el uso del SDK `boto3`, que ofrece una exposición simple a la API de Amazon Bedrock. Con esta API, generó un programa Python para analizar y visualizar determinados datos de ventas.



### Pruébelo usted mismo

- Cambie las peticiones para que se adapten al caso práctico específico y evalúe la salida de los diferentes modelos.
- Juegue con la longitud del token para comprender la latencia y la capacidad de respuesta del servicio.
- Aplique diferentes principios de ingeniería de peticiones para obtener mejores salidas.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno y continúe con la **Tarea 6**.